#### **Objective:** <br>
Predict average rating for each skincare product. <br>

#### **Method:** <br>
1. Data Exporation
    * 1815 skincare products
    * Use ydata_profiling to generate a EDA report
    * Create a network graph to see if there are any interesting networks

2. Data Preprocessing
    * Normalize size/price, and exclude products without size(oz)
    * label encode the brand
    * Remove null values from numeric columns

3. Model Building
    * Remove features that display high multi-collinearity (VIF >= 5)
    * Build a linear regression model to predict average rating with the features:  loves_count, reviews_num, size_price_ratio, brand_size, brand

4. Model Evaluation
    * Calculate MSE, RMSE, R-squared

#### **Summary:** <br>
Linear regression is not a good model for predicting ratings. The model can predict an average rating close to the dataset's mean, but the selected predictors have a minila impact on the rating prediction. There are no signficant features. Additionally, the R-squared value was low and the the RMSE was high. 

#### **Recommendation:** <br>
Predicting ratings is important to understanding customer preferences, informing marketing strategies, and driving business growth. Although this model did not perform well for rating prediction of skincare products, exploring other features for rating prediction and other models would be insightful to the business.

In [92]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.svm import SVR
from pyvis.network import Network
import statsmodels.api as sm


##### **1a. Data Exploration**
* No correlation between numeric columns
* The majority of dummy variables representing skin concerns exhibit approximately 80% or more 0 values, indicating an imbalance in the data distribution.
* Across most dummy variables representing skin concerns, there is an even distribution with around 50% of values being 0 and 50% being 1.
* Average rating = 4.2
* Average lover count = 39,181
* Average number of reviews = 539
* Average price = 63
* Average number of connections = 4

In [93]:
#read in the data
skincare_data = pd.read_csv("../results/skincare_data_cleaned.csv")

#convert item_sku from int to str
skincare_data['item_sku'] = skincare_data['item_sku'].astype(str)

display(skincare_data.head())

,item_sku,brand,product,rating,loves_count,reviews_num,price,child_sku,item_id,similar_products,...,BUMPS,INGROWNS,UNEVEN TONE,COMBINATION,DRY,NORMAL,OILY,SKIN_TYPE_UNKNOWN,brand_product_name,connections_num
0,101220,SHISEIDO,FACIAL COTTON,4.8064,143523,2913.0,16.0,1880350,P173726,"2735132,2031391,2191526,2698348,2031441",...,0,0,0,1,1,1,1,0,SHISEIDO FACIAL COTTON,3.0
1,1027465,CLINIQUE,ACNE SOLUTIONS ALL-OVER CLEARING TREATMENT OIL...,4.0181,18325,719.0,26.0,NaN,P188306,"1027473,1677137,1027507,1592831,1592856",...,0,0,0,1,1,1,1,0,CLINIQUE ACNE SOLUTIONS ALL-OVER CLEARING TREA...,4.0
2,1027473,CLINIQUE,ACNE SOLUTIONS™ CLARIFYING LOTION,4.3980,46140,1015.0,21.0,NaN,P188307,"1027465,1027507,1677137,1592831,1802321",...,0,0,0,1,1,1,1,0,CLINIQUE ACNE SOLUTIONS™ CLARIFYING LOTION,5.0
3,1027507,CLINIQUE,ACNE SOLUTIONS™ CLEANSING FOAM,4.1654,51136,1052.0,25.0,2531747,P188309,"1027473,1027465,1677137,1592831,2531747",...,0,0,0,0,1,1,0,0,CLINIQUE ACNE SOLUTIONS™ CLEANSING FOAM,5.0
4,1064062,SHISEIDO,BENEFIANCE NUTRIPERFECT NIGHT CREAM,4.3684,6099,38.0,97.0,NaN,P202935,"2234102,2234110,1723865,2733681,1723857",...,0,0,0,0,0,0,0,1,SHISEIDO BENEFIANCE NUTRIPERFECT NIGHT CREAM,4.0


In [74]:
#ydata_profile of the data
'''profile = ProfileReport(skincare_data)
profile.to_file("../results/skincare_data_EDA.html")'''


'profile = ProfileReport(skincare_data)\nprofile.to_file("../results/skincare_data_EDA.html")'

##### **1b. Network Graph of Products**

In [75]:
network_data = pd.read_csv("../results/product_network_data.csv")
network_data.head()

,from_sku,to_sku,from_name,to_name
0,101220,2735132,SHISEIDO FACIAL COTTON,THE ORDINARY HYALURONIC ACID 2% + B5 HYDRATING...
1,101220,2031391,SHISEIDO FACIAL COTTON,THE ORDINARY NIACINAMIDE 10% + ZINC 1% OIL CON...
2,101220,2031441,SHISEIDO FACIAL COTTON,THE ORDINARY ALPHA ARBUTIN 2% + HA HYPERPIGMEN...
3,1027465,1027473,CLINIQUE ACNE SOLUTIONS ALL-OVER CLEARING TREA...,CLINIQUE ACNE SOLUTIONS™ CLARIFYING LOTION
4,1027465,1677137,CLINIQUE ACNE SOLUTIONS ALL-OVER CLEARING TREA...,CLINIQUE ACNE SOLUTIONS™ CLEANSING GEL


In [76]:
#network graph of all the connections

#import the data
network_data = pd.read_csv("../results/product_network_data.csv")
network_data.head()

#get the unique nodes (sku_id)
sku_nodes = set(network_data['from_sku'])
sku_nodes.update(network_data['to_sku'])
sku_nodes = list(sku_nodes)
print("Number of Nodes: ", len(sku_nodes))

G = Network(notebook=True, cdn_resources='in_line')
#add nodes


for node_id in sku_nodes:
    node_label_df = network_data.loc[network_data['from_sku'] == node_id, 'from_name'] 
    if len(node_label_df)!=0:
        node_label = node_label_df.iloc[0]
    else:
        node_label = network_data.loc[network_data['to_sku'] == node_id, 'to_name'].iloc[0]
 
    G.add_node(node_id, label=node_label)


#add edges
edges = list(zip(network_data['from_sku'], network_data['to_sku']))
G.add_edges(edges)

G.save_graph("../results/skincare_network_graph.html")


Number of Nodes:  1777


##### **2. Data Preproccessing**
 * Normalize size/price, and exclude products without size(oz)
 * Fill in hierarchy_3 null values with Unknown
 * label encode the brand
 * Remove null values from numeric columns - rating, number of reviews, prize/size ratio, love_counts

In [94]:
skincare_data_modified = skincare_data.copy()

#Normalize size/price
skincare_data_modified['size_price_ratio'] = skincare_data_modified['price']/skincare_data_modified['size_oz']

#label encode the brand
label_encoder = LabelEncoder()
skincare_data_modified['brand_encoded'] = label_encoder.fit_transform(skincare_data_modified['brand'])



In [95]:
#for the hierarchies, check for nulls and label encode it
print("Number of null values in hierarchy_2: ", skincare_data_modified['hierarchy_2'].isna().sum())
print("Number of null values in hierarchy_3: ", skincare_data_modified['hierarchy_3'].isna().sum())

#fill in null values with "UNKNOWN" for'hierarchy_3'
skincare_data_modified['hierarchy_3'] = skincare_data_modified['hierarchy_3'].fillna("UNKNOWN")

#label encode the hierachies
skincare_data_modified['hierarchy_2_encoded'] = label_encoder.fit_transform(skincare_data_modified['hierarchy_2'])
skincare_data_modified['hierarchy_3_encoded'] = label_encoder.fit_transform(skincare_data_modified['hierarchy_3'])

Number of null values in hierarchy_2:  0
Number of null values in hierarchy_3:  235


In [96]:
#remove numeric rows with null values for ratiing, reviews_num, size_price_ratio

#remove null ratings (22 products)
skincare_data_modified.dropna(subset=['rating'], inplace=True)

#remove null review (22 products)
skincare_data_modified.dropna(subset=['reviews_num'], inplace=True)

#remove null size_price_ratio (489). These products don't have a size   
skincare_data_modified.dropna(subset=['size_price_ratio'], inplace=True) 

#remove null loves_count
skincare_data_modified.dropna(subset=['loves_count'], inplace=True)

print("Number of products in the final dataset: ", len(skincare_data_modified))

Number of products in the final dataset:  1505


In [97]:
#create a new column for the number of products per brand
skincare_data_modified['brand_size'] = skincare_data_modified.groupby('brand')['brand'].transform('size')

##### **3. Model Building**
 * Determine which features display multi-collinearity by calculating VIF. IF VIF >= 5, exclude the features 
 * Build a linear regression model to predict average rating

In [98]:
#function to calculate vif
def calc_vif(df):
    vif = pd.DataFrame()
    vif["Variable"] = df.columns
    vif["VIF"] = [variance_inflation_factor(df.values, i) 
                            for i in range(len(df.columns))] 
    return vif

In [99]:
#calculate vif for potential features
potential_features_list = ['loves_count', 'reviews_num',
                            'price',  'SKIN_CONCERN_UNKNOWN',
                            'ACNE', 'BLEMISHES', 'OILINESS', 'PORES', 'FINE LINES', 'WRINKLES',
                            'DULLNESS', 'FIRMNESS', 'ELASTICITY', 'UNEVEN TEXTURE', 'DRYNESS',
                            'MASK', 'DARK SPOTS', 'PUFFINESS', 'DARK CIRCLES', 'BUMPS', 'INGROWNS',
                            'UNEVEN TONE', 'COMBINATION', 'DRY', 'NORMAL', 'OILY',
                            'SKIN_TYPE_UNKNOWN', 'connections_num',
                            'size_price_ratio', 'brand_encoded', 'hierarchy_2_encoded',
                            'hierarchy_3_encoded', 'brand_size']

#make a new df with the potential features
potential_features_df = skincare_data_modified[potential_features_list]

#calculate the vif
potential_features_vif = calc_vif(potential_features_df)
display(potential_features_vif)

c:\Users\yong2\miniconda3\envs\projects\Lib\site-packages\statsmodels\regression\linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


,Variable,VIF
0,loves_count,2.342515
1,reviews_num,2.538671
2,price,2.814781
3,SKIN_CONCERN_UNKNOWN,16.853447
4,ACNE,30.282964
5,BLEMISHES,31.266248
6,OILINESS,1.656877
7,PORES,1.930490
8,FINE LINES,153.062135
9,WRINKLES,152.755969


In [100]:
#select feature columns, exclude features with VIF >= 5. 
#some skin types and skin concerns have high VIF, so exclude all skin type and skin concerns

selected_columns = ['rating', 'loves_count', 'reviews_num','size_price_ratio', 'brand_size', 'brand_encoded','hierarchy_2_encoded']

# Select the subset of columns
skincare_data_modified_lm = skincare_data_modified[selected_columns]

In [101]:
#calculate the vif of selected features
X_features = skincare_data_modified_lm[['loves_count', 'reviews_num','size_price_ratio', 'brand_size', 'brand_encoded','hierarchy_2_encoded']]

X_features_vif = calc_vif(X_features)

display(X_features_vif)

,Variable,VIF
0,loves_count,2.250417
1,reviews_num,2.430859
2,size_price_ratio,1.177378
3,brand_size,2.509540
4,brand_encoded,2.475670
5,hierarchy_2_encoded,2.652842


In [102]:
#split the dataset into 70% training and 30% testing
y = skincare_data_modified_lm['rating']
X = skincare_data_modified_lm.drop(['rating'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 1235)


In [103]:
# Fit and transform the training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [104]:
#build linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calculate R-squared (R2)
r2 = r2_score(y_test, y_pred)
print("R-squared (R2):", r2)

# Calculate Root Mean Squared Error (RMSE)
rmse = root_mean_squared_error(y_test, y_pred)  # squared=False to get RMSE instead of MSE
print("Root Mean Squared Error (RMSE):", rmse)

# Print model coefficients
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

Mean Squared Error: 0.15817916355380068
R-squared (R2): 0.008801664421000277
Root Mean Squared Error (RMSE): 0.3977174418526307
Coefficients: [-3.74870249e-07  3.22786359e-05 -4.83623957e-05 -2.83629680e-03
 -2.29392816e-05  3.90467283e-03]
Intercept: 4.290562683017838


In [105]:
#unscale the coefficients for interpretation
unscaled_coef = (model.coef_ * scaler.var_)+scaler.mean_
for col, coef in zip(X_train.columns, unscaled_coef):
    print(col, round(coef,2))

loves_count 38060.07
reviews_num 625.85
size_price_ratio 58.05
brand_size 22.27
brand_encoded 63.89
hierarchy_2_encoded 5.02


In [106]:
#check the statistical significance of the variables using stastmodel.api 
X_sm = sm.add_constant(X_train_scaled)

# Fit the linear regression model
sm_model = sm.OLS(y_train, X_train_scaled).fit()

# Print the summary of the regression model
print(sm_model.summary())


                                 OLS Regression Results                                
Dep. Variable:                 rating   R-squared (uncentered):                   0.000
Model:                            OLS   Adj. R-squared (uncentered):             -0.006
Method:                 Least Squares   F-statistic:                            0.02380
Date:                Mon, 26 Feb 2024   Prob (F-statistic):                        1.00
Time:                        16:10:25   Log-Likelihood:                         -3021.2
No. Observations:                1053   AIC:                                      6054.
Df Residuals:                    1047   BIC:                                      6084.
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

##### **4. Model Evaluation**
 * The intercept of the linear regression model is ~4.3. If all predictor variables, the model predicts an average rating of ~4.3. Data exploration shows that the average rating across all skincare products is also ~4.3
 * love_counts increase the prediction of the ratings the most
 * RMSE is ~0.4. This is the average prediction error of each rating.
 * R-squared shows that a very small portion of the vairaince in the average rating is explained by the predictors in the model
 * None of the predictors are significant 
 * Linear regression appears to be an inadequate model for predicting ratings based on the findings observed. 

##### **5. Save the data**

In [107]:
# Save data for the lm model to csv
skincare_data_modified_lm.to_csv('../results/skincare_data_model_input.csv', index=False)

print("Data saved to 'skincare_data_model_input.csv'")

Data saved to 'skincare_data_model_input.csv'


In [108]:
#save data from Tableau visualization
tableau_col_list = ['item_sku', 'brand', 'product', 'rating', 'loves_count', 'reviews_num',
       'price', 'child_sku', 'item_id', 'similar_products', 'size_oz',
       'hierarchy_1', 'hierarchy_2', 'hierarchy_3', 'SKIN_CONCERN_UNKNOWN',
       'ACNE', 'BLEMISHES', 'OILINESS', 'PORES', 'FINE LINES', 'WRINKLES',
       'DULLNESS', 'FIRMNESS', 'ELASTICITY', 'UNEVEN TEXTURE', 'DRYNESS',
       'MASK', 'DARK SPOTS', 'PUFFINESS', 'DARK CIRCLES', 'BUMPS', 'INGROWNS',
       'UNEVEN TONE', 'COMBINATION', 'DRY', 'NORMAL', 'OILY',
       'SKIN_TYPE_UNKNOWN', 'connections_num',
       'size_price_ratio', 'brand_size']
tableau_df = skincare_data_modified[tableau_col_list]

tableau_df.to_csv('../results/skincare_data_Tableau_input.csv', index=False)

print("Data saved to 'skincare_data_Tableau_input.csv'")

Data saved to 'skincare_data_Tableau_input.csv'
